# 파이썬 상속

## super()로 기반 클래스 초기화

In [3]:
class Person:
    def __init__(self):
        self.hello = '안녕하세요.'
        
class Student(Person):
    def __init__(self):
        self.school = '서울 소프트웨어 아카데미'
        
dp = Student()
try:
    print('1 : ', dp.school)
    print('2 : ', dp.hello)
except:
    print('다시 상속하세요')

1 :  서울 소프트웨어 아카데미
다시 상속하세요


In [13]:
class Student(Person):
    def __init__(self):
        # 상속한 클래스의 __init__ 함수를 호출해서 실행
        super().__init__()
        self.school = '서울 소프트웨어 아카데미'
        
dp = Student()

try:
    print('1 : ', dp.school)
    print('2 : ', dp.hello)
except:
    print('다시 상속하세요')

1 :  서울 소프트웨어 아카데미
2 :  안녕하세요.


## 클래스를 초기화하지 않아도 되는 경우

In [15]:
class Person:
    def __init__(self):
        self.hello = '안녕하세요.'
        
class Student(Person):
    pass # 상속받은 클래스에 __init__ 함수가 없다면 클래스 초기화를 안해줘도된다.

dp = Student()
print(dp.hello)

안녕하세요.


# 인공신경망

In [18]:
!pip install -U --no-cache-dir gdown --pre
print('Downloading...')
!gdown --id {"1NDg2_ccmxkWDu2e_2hsngYI5pzY4qWkW"} -O reg.csv

Requirement already up-to-date: gdown in c:\users\jay\appdata\local\programs\python\python37-32\lib\site-packages (4.4.0)
Downloading...


You are using pip version 19.0.3, however version 22.0.4 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.
c:\users\jay\appdata\local\programs\python\python37-32\lib\site-packages\gdown\cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1NDg2_ccmxkWDu2e_2hsngYI5pzY4qWkW
To: C:\Users\Jay\Desktop\New_git\TIL\Deep-Learning\pytorch_basic\reg.csv

  0%|          | 0.00/138k [00:00<?, ?B/s]
100%|##########| 138k/138k [00:00<00:00, 1.53MB/s]


## 모델구조 및 가중치 확인

In [21]:
import torch
from torch import nn
import torch.nn.functional as F
#!pip install torchsummary
from torchsummary import summary

In [29]:
class Regressor(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(13, 50)
        self.fc2 = nn.Linear(50, 30)
        self.fc3 = nn.Linear(30, 1)
        self.dropout = nn.Dropout()
        
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(F.relu(self.fc2(x)))
        x = F.relu(self.fc3(x))
        
        return x

In [23]:
model = Regressor()

In [25]:
print(model)

Regressor(
  (fc1): Linear(in_features=13, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=30, bias=True)
  (fc3): Linear(in_features=30, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)


In [31]:
for name, parameter in model.named_parameters():
    print(name, ': ', parameter.size())

fc1.weight :  torch.Size([50, 13])
fc1.bias :  torch.Size([50])
fc2.weight :  torch.Size([30, 50])
fc2.bias :  torch.Size([30])
fc3.weight :  torch.Size([1, 30])
fc3.bias :  torch.Size([1])


In [32]:
print(model.fc1.weight.size(), model.fc1.bias.size())

torch.Size([50, 13]) torch.Size([50])


In [34]:
device = torch.device('cuda')
print(device)

cuda


In [37]:
summary(model.to(device), (10, 13))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1               [-1, 10, 50]             700
            Linear-2               [-1, 10, 30]           1,530
           Dropout-3               [-1, 10, 30]               0
            Linear-4                [-1, 10, 1]              31
Total params: 2,261
Trainable params: 2,261
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 0.01
Estimated Total Size (MB): 0.02
----------------------------------------------------------------


# 교차 검증

In [59]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# ANN
import torch
from torch import nn, optim # torch 내의 세부적인 기능을 불러온다.
from torch.utils.data import DataLoader, Dataset # 데이터를 모델에 사용할 수 있도록 정리해 주는 라이브러리
import torch.nn.functional as F # torch 내의 세부적인 기능을 불러온다 (신경망 기술 등)

# Cross Validation
from sklearn.model_selection import KFold

# Loss
from sklearn.metrics import mean_squared_error # Regression 문제의 평가를 위해 MSE(Mean Squared Error)를 불러온다.

# Plot
import matplotlib.pyplot as plt # 시각화 도구

In [39]:
df = pd.read_csv('./reg.csv', index_col=[0])
df.head()

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,Price
0,0.034633,0.206919,0.137057,0.540526,0.193941,0.699239,0.630532,0.239410,0.027375,0.209857,0.347609,0.996394,0.102644,0.422222
1,0.028920,0.014315,0.276113,0.255945,0.618886,0.555407,0.782263,0.482977,0.103031,0.106690,0.520776,0.996650,0.187120,0.368889
2,0.020627,0.033230,0.281116,0.525591,0.165269,0.624102,0.586005,0.272713,0.036010,0.106986,0.595301,0.983284,0.084079,0.660000
3,0.022749,0.033801,0.125044,0.263253,0.251509,0.658532,0.432160,0.344932,0.150018,0.068317,0.651297,0.989989,0.015990,0.631111
4,0.022148,0.029374,0.121057,0.521126,0.399670,0.448086,0.520158,0.495342,0.104383,0.069360,0.560116,0.998723,0.092782,0.693333


In [40]:
# 데이터를 넘파이 배열로 만들기
X = df.drop('Price', axis=1).to_numpy() # 데이터프레임에서 타겟값을 제외하고 넘파이 배열로 만들기
Y = df['Price'].to_numpy().reshape((-1, 1)) # 데이터프레임 형태의 타겟값을 넘파이 배열로 만들기

In [42]:
print(X.shape, Y.shape)

(506, 13) (506, 1)


In [49]:
class TensorData(Dataset):
    def __init__(self, x_data, y_data):
        self.x_data = torch.FloatTensor(x_data)
        self.y_data = torch.FloatTensor(y_data)
        self.len = self.y_data.shape[0]
        
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]
        
    def __len__(self):
        return self.len

In [50]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.7)
trainset = TensorData(X_train, Y_train)
testset = TensorData(X_test, Y_test)

In [52]:
trainset[0] # 이런거 할 때 __getitem__없으면 호출이안됌(객체 슬라이싱)

(tensor([0.2620, 0.0083, 0.6565, 0.5732, 0.4741, 0.3701, 0.8900, 0.0000, 0.9041,
         0.9089, 0.5846, 0.7179, 0.8440]),
 tensor([0.0844]))

In [54]:
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False)
kfold = KFold(n_splits=3, shuffle=True)

In [ ]:
class Regressor(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(13, 50)
        self.fc2 = nn.Linear(50, 30)
        self.fc3 = nn.Linear(30, 1)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x

In [55]:
criterion = nn.MSELoss()

In [57]:
def evaluation(dataloader):
    predictions = torch.tensor([], dtype=torch.float)
    actual = torch.tensor([], dtype=torch.float)
    
    with torch.no_grad():
        model.eval() # 평가를 할 때에는 .eval() 반드시 써야한다.
        for data in dataloader:
            inputs, values = data
            outputs = model(inputs)
            
            predictions = torch.cat((predictions, outputs), 0) # cat을 통해 예측값을 누적
            actual = torch.cat((actual, values), 0) # cat을 통해 실제값을 누적
            
    predictions = predictions.numpy() # 넘파이 배열로 변경
    actual = actual.numpy() # 넘파이 배열로 변경
    rmse = np.sqrt(mean_squared_error(predictions, actual)) # sklearn을 이용해 RMSE 계산
    model.train()
    return rmse

# 평가 시 .eval()을 사용해야 하는 이유
# 이번 예시에서는 상관없으나 평가 시에는 정규화 기술을 배제하여 온전한 모델로 평가를 해야한다.
# 따라서 .eval()을 사용한다.
# 즉, 드랍아웃이나 배치 정규화 등과 같이 학습 시에만 사용하는 기술들이 적용된 모델은 평가 시에는 비활성화 해야하며 학습시
# .train()을 사용한다.

In [60]:
validation_loss = []

for fold, (train_idx, val_idx) in enumerate(kfold.split(trainset)):
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_idx) # index 생성
    val_subsampler = torch.utils.data.SubsetRandomSampler(val_idx) # index 생성

    # sampler를 이용한 DataLoader 정의
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, 
                                              sampler=train_subsampler) 
    valloader = torch.utils.data.DataLoader(trainset, batch_size=32, 
                                            sampler=val_subsampler)
    
    # 모델
    model = Regressor()
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-7)

    for epoch in range(400): # 400번 학습을 진행한다.
        for data in trainloader: # 무작위로 섞인 32개 데이터가 있는 배치가 하나 씩 들어온다.
            inputs, values = data # data에는 X, Y가 들어있다.
            optimizer.zero_grad() # 최적화 초기화

            outputs = model(inputs) # 모델에 입력값 대입 후 예측값 산출
            loss = criterion(outputs, values) # 손실 함수 계산
            loss.backward() # 손실 함수 기준으로 역전파 설정 

            optimizer.step() # 역전파를 진행하고 가중치 업데이트

    train_rmse = evaluation(trainloader) # 학습 데이터의 RMSE
    val_rmse = evaluation(valloader)
    print("k-fold", fold," Train Loss: %.4f, Validation Loss: %.4f" %(train_rmse, val_rmse)) 
    validation_loss.append(val_rmse)

validation_loss = np.array(validation_loss)
mean = np.mean(validation_loss)
std = np.std(validation_loss)
print("Validation Score: %.4f, ± %.4f" %(mean, std))   

k-fold 0  Train Loss: 0.0554, Validation Loss: 0.0801
k-fold 1  Train Loss: 0.0607, Validation Loss: 0.0740
k-fold 2  Train Loss: 0.4050, Validation Loss: 0.4650
Validation Score: 0.2064, ± 0.1829


In [ ]:
optimizer = optim.Adam(model.parameters())